In [1]:
import importlib


import os
from traceback import print_tb
import pandas as pd
import duckdb

import os, sys 
from pathlib import Path
tmp = Path.cwd()
print(tmp)
os.chdir(Path.cwd().parent)


if Path.cwd() not in sys.path:
 sys.path.append(Path.cwd())

import utils


os.chdir(tmp)
#print(tmp)

/Users/miona.dimic/Desktop/MDS/ADSDB/project/mimi-adsdb/Notebooks/3.Trusted zone


In [2]:


con = duckdb.connect('../nationalities.duckdb')

nat2018 = con.execute("SELECT *  FROM nationalities2018 LIMIT 0").df()
natcols_2018 = nat2018.columns

nat2019 = con.execute("SELECT * FROM nationalities2019 LIMIT 0").df()
natcols_2019 = nat2019.columns

nat2020 = con.execute("SELECT * FROM nationalities2020 LIMIT 0").df()
natcols_2020 = nat2020.columns 

natJoined = con.execute("SELECT * FROM nationalities LIMIT 0").df()
cols = natJoined.columns 

#checking for possible typos inside nationalities table

# missing in 2018
dif18= cols.difference(natcols_2018)

# missing in 2019
dif19 = cols.difference(natcols_2019)

#missing in 2020
dif20 = cols.difference(natcols_2020)

print(dif18)
print(dif19)
print(dif20)

#joined table has both Belarús and Bielorrusia, República_Democrática_del_Congo and República_Democrática_del_Cong

Index(['Belarús', 'Mónaco', 'República_Democrática_del_Congo', 'Samoa',
       'Santa_Sede', 'Year'],
      dtype='object')
Index(['Bielorrusia', 'Djibouti', 'Islas_Marshall',
       'República_Democrática_del_Congo', 'Surinam', 'Vanuatu', 'Year'],
      dtype='object')
Index(['Bielorrusia', 'Islas_Marshall', 'Papúa_Nueva_Guinea',
       'República_Democrática_del_Cong', 'Samoa',
       'San_Vicente_y_Las_Granadinas', 'Surinam', 'Year'],
      dtype='object')


In [3]:
# remove duplicates
con = duckdb.connect('../nationalities.duckdb')
n2018 = con.execute("SELECT *  FROM nationalities WHERE  Year = 2018").df()
n2018 = n2018[n2018.columns.difference(['Belarús','República_Democrática_del_Congo' ])] # remove empty columns Belarús and República_Democrática_del_Congo
n2018 = n2018.rename(columns = {'Bielorrusia':'Belarús', 'República_Democrática_del_Cong':'República_Democrática_del_Congo'})  #rename in right format

n2019 = con.execute("SELECT *  FROM nationalities WHERE  Year = 2019").df()
n2019 = n2019[n2019.columns.difference(['Bielorrusia','República_Democrática_del_Cong' ])] # remove empty columns Bielorrusia and República_Democrática_del_Cong


n2020 = con.execute("SELECT *  FROM nationalities WHERE  Year = 2020").df() 
n2020 = n2020[n2020.columns.difference(['Bielorrusia','República_Democrática_del_Cong' ])] # remove empty columns  Bielorrusia and República_Democrática_del_Cong

cleanNatTable = pd.concat([n2018, n2019, n2020],ignore_index = True)

#display(cleanNatTable)

utils.df_to_DBtable('../nationalities.duckdb',cleanNatTable, 'nationalitiesClean')

con.close()


In [4]:
con = duckdb.connect('../nationalities.duckdb')
df = con.execute(f'SELECT Madrid_section, year, COUNT(*) FROM nationalitiesClean  GROUP BY Madrid_section , year HAVING COUNT(*) > 1').df()
con.close()
display(df)
# no duplicates for section-code found

,Madrid_section,Year,count_star()
